<a href="https://colab.research.google.com/github/ordevoir/Data_Analysis/blob/main/pandas_prepare_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# housing = pd.read_csv('https://raw.githubusercontent.com/ordevoir/Data_Analysis/main/housing.csv')
housing = pd.read_csv('housing.csv')

In [15]:
housing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   median_income       20640 non-null  float64
 8   median_house_value  20640 non-null  float64
 9   ocean_proximity     20640 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


# Решение проблемы с отсутствующими признаками

Информация об данных показывает, что у многих образцов отсутствуют значения признака `total_bedrooms`.

Большинство алгоритмов машинного обучения не могут работать с отсутствующими значениями признаков. В нашем случае мы видим, что у многих образцов отсутствуют значения признака `total_bedrooms`. Можно исправить ситуацию несколькими способами:

1. избавиться от образцов, у которых отсутствуют признаки `total_bedrooms`:
    
    `housing.dropna(subset=["total_bedrooms"])`<br>

2. избавиться от самого атрибута total_bedrooms:

    `housing.drop("total_bedrooms", axis=1)`

3. заполнить отсутствующие признаки значениями (нулями, средним арифметическим, и тд.):

    `mean = housing["total_bedrooms"].mean()`

    `housing["total_bedrooms"].fillna(mean)`

в последнем случае значение `mean` следует сохранить до конца вычислений, так как для заполненных данных пересчет этого значения может дать уже другое значение.

> Окончание -`na` в этих методав является аббревиатурой *Not Available* (нет в наличии). Такие методы оказывают влияние только на те ячейки таблицы данных, в которых отсутствуют значения.

## Удаление образцов

Удалим образцы с отсутствующими значениями признака `total_bedrooms`.

In [16]:
cleared_housing = housing.dropna(subset=["total_bedrooms"])

In [17]:
cleared_housing.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20433 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20433 non-null  float64
 1   latitude            20433 non-null  float64
 2   housing_median_age  20433 non-null  float64
 3   total_rooms         20433 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20433 non-null  float64
 6   households          20433 non-null  float64
 7   median_income       20433 non-null  float64
 8   median_house_value  20433 non-null  float64
 9   ocean_proximity     20433 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.7+ MB


## Удаление признака

Удалим признак `total_bedrooms`. Параметр `axis` определяет работаем ли мы со строками или с колонками. Значение `axis=0` соответствует строкам (образцам) а `axis=1` – колонкам (признакам).

In [18]:
cleared_housing = housing.drop(labels=["total_bedrooms", "ocean_proximity"], axis=1)

In [19]:
cleared_housing.head()

,longitude,latitude,housing_median_age,total_rooms,population,households,median_income,median_house_value
0,-122.23,37.88,41.0,880.0,322.0,126.0,8.3252,452600.0
1,-122.22,37.86,21.0,7099.0,2401.0,1138.0,8.3014,358500.0
2,-122.24,37.85,52.0,1467.0,496.0,177.0,7.2574,352100.0
3,-122.25,37.85,52.0,1274.0,558.0,219.0,5.6431,341300.0
4,-122.25,37.85,52.0,1627.0,565.0,259.0,3.8462,342200.0


## Заполнение отсутствующих ячеек

Получим копию исходной таблицы данных в переменную `filled_housing`. Вычислим среднее значение и медиану признака `total_bedrooms` при помощи методов `mean()` и `median()`.

In [20]:
filled_housing = housing.copy(deep=True)            # создается копия данных
mean = filled_housing["total_bedrooms"].mean()      # получаем среднее
median = filled_housing["total_bedrooms"].median()  # получаем медиану
mean, median

(537.8705525375618, 435.0)

Заполненим отсутствующие ячейки признака `total_bedrooms` значением `mean` используя метод `fillna()`. Метод возвращает объект класса `Series`, который можно присвоить колонке в объекте `DataFrame`. Исходный объект, у которого вызывается метод, при этом не меняется.

In [21]:
type(filled_housing["total_bedrooms"].fillna(mean))

pandas.core.series.Series

Если задать в методе параметр `inplace=True`, то произойдет заполнение значениями на месте, в колонке самого объекта. Метод при этом ничего не будет возвращать (`None`).

In [22]:
# filled_housing["total_bedrooms"] = filled_housing["total_bedrooms"].fillna(mean)
filled_housing["total_bedrooms"].fillna(mean, inplace=True)

# Конвертация категориальных данных

Единственный текстовый атрибут в нашей выборке – это `ocean_proximity`.
Это не произвольные текстовые значения, возможные значения ограничены, поэтому здесь имеем дело с категоральным атрибутом. Большинство алгоритмов предпочитают работать с числами, поэтому целесообразно конвертировать эти категории из текстовой формы в числовую.

In [23]:
housing_cat = housing[["ocean_proximity"]]
print(housing_cat.value_counts())

ocean_proximity
<1H OCEAN          9136
INLAND             6551
NEAR OCEAN         2658
NEAR BAY           2290
ISLAND                5
Name: count, dtype: int64


In [24]:
housing_cat.head()

,ocean_proximity
0,NEAR BAY
1,NEAR BAY
2,NEAR BAY
3,NEAR BAY
4,NEAR BAY


## [`OrdinalEncoder`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OrdinalEncoder.html)

Для конвертации категориальных значений в числовые, можно использовать класс `OrdinalEncoder` из модуля `preprocessing` библиотеки `sklearn`. Для этого создается экземпляр класса `OrdinalEncoder`. Вызывается метод `fit()`, который возвращает массив числовых значений, соответствующие категориям.

In [25]:
from sklearn.preprocessing import OrdinalEncoder

ordinal_encoder = OrdinalEncoder(dtype=np.float16)
ordinal_encoder.fit(housing_cat)
housing_cat_encoded = ordinal_encoder.transform(housing_cat)
type(housing_cat_encoded), housing_cat_encoded[:3]

(numpy.ndarray,
 array([[3.],
        [3.],
        [3.]], dtype=float16))

In [26]:
housing_cat_encoded.shape

(20640, 1)

Нзавания категорий хранятся в поле `categories_` объекта класса `OrdinalEncoder` в виде массива, а числовые значения, кодирующие категории, соответствуют индексам этих категорий.

In [27]:
ordinal_encoder.categories_

[array(['<1H OCEAN', 'INLAND', 'ISLAND', 'NEAR BAY', 'NEAR OCEAN'],
       dtype=object)]

Проблемой такого представления является то, что алгоритм может посчитать, что близкие значения параметра, например 0 и 1, свидетельствуют о том, что они ближе, чем, скажем, 0 и 4, что не соответствует действительности. Чтобы избежать таких проблем, можно создавать бинарный атрибут для категорий (*one-hot encoding*) при помощи класса `OneHotEncoder`.

## [`OneHotEncoder`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html)

Интерфейс подобен классу `OrdinalEncoder`, однако, метод `transorm()` возвращает разареженную матрицу (объект класса `csr_encoded`). Для того, чтобы получить матрицу NumPy, можно воспользоваться методом `toarray()`.

In [28]:
from sklearn.preprocessing import OneHotEncoder

cat_encoder = OneHotEncoder(sparse_output=False)
housing_cat_encoded = cat_encoder.fit_transform(housing_cat)
housing_cat_encoded[:5]

array([[0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.]])

In [29]:
cat_encoder.categories_[0]

array(['<1H OCEAN', 'INLAND', 'ISLAND', 'NEAR BAY', 'NEAR OCEAN'],
      dtype=object)

Стоит иметь в виду, что если категорий очень много, то матрица NumPy может оказаться слишком расточительной, гораздо рациональнее будет использовать разреженную матрицу. Либо попробовать заменить признак не его упорядоченное представление. Например для `ocean_proximity` заменить на расстояние до океана.

## Замена категориальных признаков

Мы уже получили кодированные признаки для категориального признака `ocean_proximity`. Заменим этот признак на полученные значения в `housing_cat_encoded`.

In [30]:
# housing.drop("ocean_proximity", axis=1, inplace=True)
housing[cat_encoder.categories_[0]] = housing_cat_encoded

In [31]:
cat_encoder.categories_[0]

array(['<1H OCEAN', 'INLAND', 'ISLAND', 'NEAR BAY', 'NEAR OCEAN'],
      dtype=object)

In [32]:
cat_encoder.feature_names_in_

array(['ocean_proximity'], dtype=object)

In [33]:
housing.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity,<1H OCEAN,INLAND,ISLAND,NEAR BAY,NEAR OCEAN
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY,0.0,0.0,0.0,1.0,0.0
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY,0.0,0.0,0.0,1.0,0.0
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY,0.0,0.0,0.0,1.0,0.0
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY,0.0,0.0,0.0,1.0,0.0
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY,0.0,0.0,0.0,1.0,0.0


# Отделение целевого признака

Извлекаем данные целевого признака в виде одномерного массива NumPy и записываем их в переменную `y`. Остальные признаки извлекаем в виде двумерного массива NumPy и записываем в `X`. Для этого можно использовать метод `to_numpy()`.

In [34]:
X = cleared_housing.drop(columns=['median_house_value']).to_numpy()
y = cleared_housing['median_house_value'].to_numpy()                # целевой признак

In [35]:
print(X[:3])

[[-122.23     37.88     41.      880.      322.      126.        8.3252]
 [-122.22     37.86     21.     7099.     2401.     1138.        8.3014]
 [-122.24     37.85     52.     1467.      496.      177.        7.2574]]


In [36]:
print(y[:3])

[452600. 358500. 352100.]


# Масштабирование данных

In [37]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X)
X_scaled = scaler.transform(X)
X_scaled

array([[-1.32783522,  1.05254828,  0.98214266, ..., -0.9744286 ,
        -0.97703285,  2.34476576],
       [-1.32284391,  1.04318455, -0.60701891, ...,  0.86143887,
         1.66996103,  2.33223796],
       [-1.33282653,  1.03850269,  1.85618152, ..., -0.82077735,
        -0.84363692,  1.7826994 ],
       ...,
       [-0.8237132 ,  1.77823747, -0.92485123, ..., -0.3695372 ,
        -0.17404163, -1.14259331],
       [-0.87362627,  1.77823747, -0.84539315, ..., -0.60442933,
        -0.39375258, -1.05458292],
       [-0.83369581,  1.75014627, -1.00430931, ..., -0.03397701,
         0.07967221, -0.78012947]])

In [39]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(housing, test_size=0.2, random_state=42)
print(type(train_set))  # типы данных DataFrame
print(len(train_set), len(test_set))

<class 'pandas.core.frame.DataFrame'>
16512 4128


# Разбиение данных на тренировочную и тестовую наборы

In [40]:
import numpy as np

def split_train_test(data, test_ratio):
    shuffled_indices = np.random.permutation(len(data))
    test_set_size = int(len(data) * test_ratio)
    test_indices = shuffled_indices[:test_set_size]
    train_indices = shuffled_indices[test_set_size:]
    return data.iloc[train_indices], data.iloc[test_indices]

In [44]:
train, test = split_train_test(cleared_housing, 0.2)
len(train), len(test)

(16512, 4128)

## Функция `train_test_split()`

Функция `train_test_split()` может производить разбиение как для объектов класса `DataFrame` и `Series`, так и для массивов NumPy:

In [53]:
from sklearn.model_selection import train_test_split

X_train, X_test = train_test_split(X, test_size=0.2, random_state=42)
y_train, y_test = train_test_split(y, test_size=0.2, random_state=42)
print(type(X_train))  # типы данных DataFrame
print(len(X_train), len(X_test))

<class 'numpy.ndarray'>
16512 4128


In [57]:
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=2)

len(X_train), len(X_test), len(y_train), len(y_test)

(16512, 4128, 16512, 4128)